## Import Libraries

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM
from keras.utils import to_categorical
from pickle import dump, load
from nltk.corpus import words
import nltk
import spacy
import keras

import numpy as np
import PyPDF2
import random
import re

nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jayaw\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

## Open and Convert PDF File to Python String

In [6]:
with open('the_psychology_of_money_1.pdf', 'rb') as f: 
    pdf_reader = PyPDF2.PdfReader(f)

    text  = ''
    for i in pdf_reader.pages:
        text += i.extract_text()

print(text)

Machine Translated by GoogleMachine Translated by GoogleMachine Translated by GoogleGretchen,  yang  membimbingku.Untuk
Miles  dan  Reese,  yang  menginspirasi  saya.Orang  tuaku,  yang  mengajariku.Machine Translated by GooglePendahuluan:  Pertunjukan  Terhebat  Di  Dunia
2.  Keberuntungan  &  Risiko
3.  Tidak  Pernah  Cukup1.  Tidak  Ada  yang  GilaMachine Translated by Google4.  Peracikan  Perancu
8.  Manusia  dalam  Paradoks  Mobil
12.  Kejutan!5.  Menjadi  Kaya  vs.  Tetap  Kaya
14.  Anda  akan  Berubah9.  Kekayaan  adalah  Apa  yang  Tidak  Anda  Lihat
13.  Ruang  untuk  Kesalahan6.  Ekor,  Kamu  Menang
10.  Menghemat  Uang7.  Kebebasan
11.  Wajar  >  RasionalMachine Translated by Google19.  Semua  Bersama  Sekarang15.  Tidak  Ada  yang  Gratis
Catatan  akhir
Ucapan  Terima  Kasih16.  Kamu  &  Aku
Detail  penerbitan20.  Pengakuan17.  Rayuan  Pesimisme
Catatan  Tambahan:  Sejarah  Singkat  Mengapa  Konsumen  AS  Berpikir  Seperti  Itu18.  Kapan  Anda  Akan  Percaya  Apa  PunMachin

In [49]:
nlp = spacy.load('en_core_web_lg')
nlp.max_length = 1198623

## Data / Text Cleaning

In [68]:
sentences = text.split('.')
print('Number of sentences:', len(sentences), end='\n\n')
[i for i in sentences]

Number of sentences: 1



['dongare, rkharde, amit dkachareabstract : - this paper presents an emergence of an artificialneural network (ann) as a tool for analysis of differentparameters of a systeman artificial neural network (ann) isan information -processing paradigm that is inspired by the waybiological nervous systems such as brain, process informationann consists of multiple layers of simple processing elementscalled as neuronsthe neuron performs two functions, namel y,collection of inputs & generation of an outputuse of annprovides overview of the theory, learning rules, and applicationsof the most important neural network models, definitions andstyle of computationthe mathematical model of networkthrows the light on the concept of inputs, weights, summingfunction, activation function & outputsthen ann helps todecide the type of learning for  adjustment of weights withchange in parametersfinally the analysis of a system iscompleted by ann implementation  & ann training & predictionqualitykeywords:  biol

In [69]:
def clean_text(txt: list[str]):
    temp = []
    for i in txt:
        if not i.isspace():
            temp.append(i.strip().lower())
    return temp

In [70]:
sentences = clean_text(sentences)
print('Number of sentences:', len(sentences), end='\n\n')
sentences

Number of sentences: 1



['dongare, rkharde, amit dkachareabstract : - this paper presents an emergence of an artificialneural network (ann) as a tool for analysis of differentparameters of a systeman artificial neural network (ann) isan information -processing paradigm that is inspired by the waybiological nervous systems such as brain, process informationann consists of multiple layers of simple processing elementscalled as neuronsthe neuron performs two functions, namel y,collection of inputs & generation of an outputuse of annprovides overview of the theory, learning rules, and applicationsof the most important neural network models, definitions andstyle of computationthe mathematical model of networkthrows the light on the concept of inputs, weights, summingfunction, activation function & outputsthen ann helps todecide the type of learning for  adjustment of weights withchange in parametersfinally the analysis of a system iscompleted by ann implementation  & ann training & predictionqualitykeywords:  biol

In [62]:
def filter_sentences(txt: list[str]) -> list[str]:

    temp = []
    
    for i in txt:
        if i.lower() == 'the authors would like to acknowledge & thanks to dr.':
            break
        
        patterns = [
            r"^[iv]+.",
            r"issn:.*",
            r"iso.*certified$",
            r"international journal of engineering and innovative technology \(ijeit\)",
            r"volume \d, issue \d, .+",
            r"^\S+$",
            r"^fig\.\s?\d+.+",
            r"^table\s?\d+\..+",
            r"^[a-z]+\.",
            r"^\d+ \..+",
            r"^\d+\. .+",
        ]
        
        if any(re.search(pattern, i, re.IGNORECASE) for pattern in patterns):
            continue
        
        temp.append(i)
    
    return temp


In [63]:
sentences = filter_sentences(sentences)
print('Number of sentences:', len(sentences), end='\n\n')
sentences

Number of sentences: 336



['dongare, rkharde, amit dkachare',
 'abstract : - this paper presents an emergence of an artificial',
 'neural network (ann) as a tool for analysis of different',
 'parameters of a systeman artificial neural network (ann) is',
 'an information -processing paradigm that is inspired by the way',
 'biological nervous systems such as brain, process informationann consists of multiple layers of simple processing elements',
 'called as neuronsthe neuron performs two functions, namel y,',
 'collection of inputs & generation of an outputuse of ann',
 'provides overview of the theory, learning rules, and applications',
 'of the most important neural network models, definitions and',
 'style of computationthe mathematical model of network',
 'throws the light on the concept of inputs, weights, summing',
 'function, activation function & outputsthen ann helps to',
 'decide the type of learning for  adjustment of weights with',
 'change in parametersfinally the analysis of a system is',
 'complet

In [64]:
def replace_patterns(txt):
    patterns = [
        r'in\sfig\.\s\d',
        r'fig\.\s\d',
        r'\s?\[\s?\d+\s?]\s?',
    ]
    for pattern in patterns:
        txt = re.sub(pattern, '', txt)

    return txt

def replace_patterns_in_sentences(sentences):
    new_sentences = []
    for sentence in sentences:
        new_sentence = replace_patterns(sentence)
        new_sentences.append(new_sentence)
    return new_sentences

In [65]:
sentences = replace_patterns_in_sentences(sentences)
sentences

['dongare, rkharde, amit dkachare',
 'abstract : - this paper presents an emergence of an artificial',
 'neural network (ann) as a tool for analysis of different',
 'parameters of a systeman artificial neural network (ann) is',
 'an information -processing paradigm that is inspired by the way',
 'biological nervous systems such as brain, process informationann consists of multiple layers of simple processing elements',
 'called as neuronsthe neuron performs two functions, namel y,',
 'collection of inputs & generation of an outputuse of ann',
 'provides overview of the theory, learning rules, and applications',
 'of the most important neural network models, definitions and',
 'style of computationthe mathematical model of network',
 'throws the light on the concept of inputs, weights, summing',
 'function, activation function & outputsthen ann helps to',
 'decide the type of learning for  adjustment of weights with',
 'change in parametersfinally the analysis of a system is',
 'complet

In [66]:
text = ''.join(sentences)
text

'dongare, rkharde, amit dkachareabstract : - this paper presents an emergence of an artificialneural network (ann) as a tool for analysis of differentparameters of a systeman artificial neural network (ann) isan information -processing paradigm that is inspired by the waybiological nervous systems such as brain, process informationann consists of multiple layers of simple processing elementscalled as neuronsthe neuron performs two functions, namel y,collection of inputs & generation of an outputuse of annprovides overview of the theory, learning rules, and applicationsof the most important neural network models, definitions andstyle of computationthe mathematical model of networkthrows the light on the concept of inputs, weights, summingfunction, activation function & outputsthen ann helps todecide the type of learning for  adjustment of weights withchange in parametersfinally the analysis of a system iscompleted by ann implementation  & ann training & predictionqualitykeywords:  biolo

## Tokenization

In [67]:
# Create tokens from the document
tokens = [token.text for token in nlp(text)]
print("Number of tokens:", len(tokens))
tokens

Number of tokens: 3083


['dongare',
 ',',
 'rkharde',
 ',',
 'amit',
 'dkachareabstract',
 ':',
 '-',
 'this',
 'paper',
 'presents',
 'an',
 'emergence',
 'of',
 'an',
 'artificialneural',
 'network',
 '(',
 'ann',
 ')',
 'as',
 'a',
 'tool',
 'for',
 'analysis',
 'of',
 'differentparameters',
 'of',
 'a',
 'systeman',
 'artificial',
 'neural',
 'network',
 '(',
 'ann',
 ')',
 'isan',
 'information',
 '-processing',
 'paradigm',
 'that',
 'is',
 'inspired',
 'by',
 'the',
 'waybiological',
 'nervous',
 'systems',
 'such',
 'as',
 'brain',
 ',',
 'process',
 'informationann',
 'consists',
 'of',
 'multiple',
 'layers',
 'of',
 'simple',
 'processing',
 'elementscalled',
 'as',
 'neuronsthe',
 'neuron',
 'performs',
 'two',
 'functions',
 ',',
 'namel',
 'y',
 ',',
 'collection',
 'of',
 'inputs',
 '&',
 'generation',
 'of',
 'an',
 'outputuse',
 'of',
 'annprovides',
 'overview',
 'of',
 'the',
 'theory',
 ',',
 'learning',
 'rules',
 ',',
 'and',
 'applicationsof',
 'the',
 'most',
 'important',
 'neural',
 